In [13]:
import numpy as np
import pandas as pd
import keras
import random
import tensorflow as tf
from sklearn import preprocessing
from sklearn import metrics
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [14]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

In [15]:
path = r"C:\Users\mata2\Desktop\master\podaci\20k\Y"

In [16]:
dir = os.listdir(path)

In [17]:
file_names = []
for file in dir:
    file_names.append(file)

In [18]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    if current_file[0][i] < min_kord[i]:
      min_kord[i] = current_file[0][i]
    elif current_file[0][i] > max_kord[i]:
      max_kord[i] = current_file[0][i]

In [19]:
kord_sum = [0 for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    kord_sum[i] += current_file[0][i]

napravljene su prosecne koordinate

In [20]:
average_kord = [x//len(file_names) for x in kord_sum]

In [21]:
synth_path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\sinteticki podaci\20k\Y"

In [22]:
dir2 = os.listdir(synth_path)

In [23]:
file_names_synt = []
i = 0
for file in dir2:
    i += 1
    if i > 100:
        break       
    else:
        file_names_synt.append(file)

In [24]:
train_synt = np.concatenate([pd.read_csv(synth_path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names_synt], axis=0)
test_synt = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names], axis=0)

In [25]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [26]:
time_steps = 1

In [27]:
train_X, train_y = create_timeseries_split(train_synt, time_steps)

In [28]:
test_X, test_y = create_timeseries_split(test_synt, time_steps)

In [29]:
input_shape = (time_steps,1)

In [30]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 64, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)


In [31]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=4, batch_size = 2048, verbose = 2)

Epoch 1/4

2979/2979 - 24s - loss: 834647104.0000 - 24s/epoch - 8ms/step
Epoch 2/4
2979/2979 - 20s - loss: 1343.3610 - 20s/epoch - 7ms/step
Epoch 3/4
2979/2979 - 21s - loss: 1343.7993 - 21s/epoch - 7ms/step
Epoch 4/4
2979/2979 - 18s - loss: 1345.7571 - 18s/epoch - 6ms/step


In [32]:
train_scores = modelMSE.evaluate(test_X, test_y)

57188/57188 [==============================] - 67s 1ms/step - loss: 1338.9446


In [33]:
test_predict_MSE = modelMSE.predict(test_X)

57188/57188 [==============================] - 63s 1ms/step


In [35]:
uspeli_indeksiMSE = []
num_correct_MSE = 0

for i in range(61000):
    if(min_kord[i] <= test_predict_MSE[i][0] <= max_kord[i]):
        num_correct_MSE += 1
        uspeli_indeksiMSE.append(i)

print("Procenat koliko MSE upada u pojas", num_correct_MSE/61000)
print("Koliko je MSE upalo u pojas", num_correct_MSE)

Procenat koliko MSE upada u pojas 0.9213606557377049
Koliko je MSE upalo u pojas 56203


In [37]:
for i in range(61000):
    if not(min_kord[i] <= int(np.round(test_predict_MSE[i][0])) <= max_kord[i]):
        print(min_kord[i], int(np.round(test_predict_MSE[i][0])), max_kord[i])

0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 4
264 284 282
1097 1096 1111
1147 1144 1155
2794 2793 2803
2846 2845 2854
2900 2899 2908
3056 3055 3064
3104 3103 3111
3247 3246 3254
3299 3296 3306
3351 3348 3358
3404 3402 3411
3455 3454 3462
3506 3505 3513
3555 3554 3563
3802 3800 3809
3853 3851 3861
3904 3901 3911
3954 3951 3961
4004 4001 4011
4053 4049 4060
4102 4098 4109
4150 4149 4159
4200 4199 4209
4302 4300 4310
4353 4351 4360
4402 4401 4410
4602 4601 4610
4801 4800 4810
4850 4849 4858
4899 4898 4906
5049 5048 5058
5301 5300 5309
5398 5397 5406
5899 5898 5904
6051 6050 6058
6301 6300 6309
6351 6349 6358
6401 6398 6407
6900 6899 6906
7401 7400 7406
146548 146562 146561
146598 146614 146611
146647 146662 146661
170286 170296 170295
170262 170273 170271
172029 172037 172036
168027 168036 168035
166030 166

149511 149526 149523
149563 149580 149577
149612 149630 149628
179974 180012 180009
179978 180010 180007
178033 178042 178040
177966 177987 177984
177816 177844 177842
177792 177809 177805
177765 177773 177772
174512 174525 174522
174562 174579 174574
174612 174629 174626
175010 175022 175020
179986 180000 179998
179980 180002 180001
179983 180003 180002
179990 180002 180000
179990 180002 180000
179990 180002 180001
179990 180002 180001
179990 180002 180001
179992 180002 180001
179993 180002 180001
179993 180002 180001
179993 180002 180001
179993 180002 180001
179993 180002 180001
179993 180002 180001
179994 180002 180001
179995 180002 180001
179995 180002 180001
179994 180001 180000
179994 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002 180001
179994 180001 180000
179994 180001 180000
179994 180001 180000
179995 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002 180001
179995 180002